In [11]:
import os
import os.path
import time

import fakeyou
import pandas as pd
import requests

from tqdm.autonotebook import tqdm

# FakeYou

In [12]:
SAVE_FOLDER = os.path.join('voice_packs', 'Queen_Elizabeth', 'wav')
os.makedirs(SAVE_FOLDER, exist_ok=True)

VOICE_ID = 'TM:4jhmevqnrqp5' # https://fakeyou.com/tts/TM:4hqn3k5v1060
FY_USER=''
FY_PWD=''

In [13]:
df = pd.read_csv('./sound_list.csv', names=['file', 'text'])
fy = fakeyou.FakeYou()

In [4]:
fy.login(
    username=FY_USER, 
    password=FY_PWD
)

In [5]:
def text2wav(text: str, file_name: str):
    if not text.endswith('.') and not text.endswith('!'):
        text += '!'
    res = fy.make_tts_job(text=text, ttsModelToken=VOICE_ID)
    wav = fy.tts_poll(res)
    wav.save(os.path.join(SAVE_FOLDER, file_name))

In [6]:
df_select = df[116:]
for i, row in tqdm(df_select.iterrows(), total=len(df_select)):
    while True:
        try:
            file = row['file'].replace('.ogg', '.wav')
            text2wav(text=row['text'], file_name=file)
            break
        except Exception as e:
            print(f'Failed, {e}. Retrying in 5sec')
            time.sleep(5)

  0%|          | 0/39 [00:00<?, ?it/s]

Failed, Too many requests, try again later.. Retrying in 5sec
Failed, Too many requests, try again later.. Retrying in 5sec
Failed, check token and text/file, or try again. Retrying in 5sec
Failed, Too many requests, try again later.. Retrying in 5sec
Failed, Too many requests, try again later.. Retrying in 5sec
Failed, check token and text/file, or try again. Retrying in 5sec
Failed, Too many requests, try again later.. Retrying in 5sec
Failed, Too many requests, try again later.. Retrying in 5sec
Failed, check token and text/file, or try again. Retrying in 5sec


# ElevenLabs

In [14]:
VOICE_ID = ''
EL_TOKEN = ''

In [17]:
def generate_elevenlabs_voice(txt: str, path: str):
    CHUNK_SIZE = 1024
    url = f"https://api.elevenlabs.io/v1/text-to-speech/{VOICE_ID}"
    
    headers = {
      "Accept": "audio/mpeg",
      "Content-Type": "application/json",
      "xi-api-key": EL_TOKEN
    }
    
    data = {
      "text": txt,
      "model_id": "eleven_multilingual_v2",
      "voice_settings": {
        "stability": 0.5,
        "similarity_boost": 0.5,
        "use_speaker_boost": True
      }
    }
    
    response = requests.post(url, json=data, headers=headers)

    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=CHUNK_SIZE):
            if chunk:
                f.write(chunk)

In [19]:
df_select = df[:]
for i, row in tqdm(df_select.iterrows(), total=len(df_select)):
    while True:
        try:
            file = row['file'].replace('.ogg', '.wav')
            generate_elevenlabs_voice(
                row['text'], 
                os.path.join('voice_packs', 'Queen_Elizabeth', 'wav', file)
            )
            break
        except Exception as e:
            print(f'Failed, {e}. Retrying in 5sec')
            time.sleep(5)

  0%|          | 0/155 [00:00<?, ?it/s]